# Imports and data loading

In [1]:
import sourmash
import screed
import os
import re
import pandas as pd

import glob
from tqdm import tqdm
from joblib import Parallel, delayed

from kmer_utils import get_encoded_kmer_hashvals

%load_ext autoreload
%autoreload 2

### Read in human-mouse homologs

In [2]:
human_mouse_homologs = pd.read_csv('HOM_MouseHumanSequence.rpt', sep='\t')
print(human_mouse_homologs.shape)
human_mouse_homologs.head()

(43117, 13)


,DB Class Key,Common Organism Name,NCBI Taxon ID,Symbol,EntrezGene ID,Mouse MGI ID,HGNC ID,OMIM Gene ID,Genetic Location,"Genomic Coordinates (mouse: , human: )",Nucleotide RefSeq IDs,Protein RefSeq IDs,SWISS_PROT IDs
0,39806032,"mouse, laboratory",10090,Gdnf,14573,MGI:107430,NaN,NaN,Chr15 3.8 cM,Chr15:7840327-7867056(+),"NM_010275,NM_001301333,NM_001301357,NM_001301332","NP_001288262,NP_034405,NP_001288286,NP_001288261",P48540
1,39806032,human,9606,GDNF,2668,NaN,HGNC:4232,OMIM:600837,Chr5 p13.2,Chr5:37812677-37840044(-),"NM_199231,NM_001278098,NM_001190469,NM_000514,...","NP_001177398,NP_000505,NP_001177397,XP_0168648...",P39905
2,39806033,"mouse, laboratory",10090,Npy4r,19065,MGI:105374,NaN,NaN,Chr14 20.8 cM,Chr14:33867603-33874376(-),NM_008919,NP_032945,Q61041
3,39806033,human,9606,NPY4R,5540,NaN,HGNC:9329,OMIM:601790,Chr10 q11.22,Chr10:46458551-46470668(-),"NM_001278794,NM_005972","NP_005963,XP_011538238,XP_011538239,XP_0168718...",NaN
4,39806034,"mouse, laboratory",10090,Evx2,14029,MGI:95462,NaN,NaN,Chr2 44.13 cM,Chr2:74483335-74489901(-),"XM_006498728,XM_006498729,NM_007967","XP_006498792,NP_031993,XP_006498791",P49749


## Set output directory

In [3]:
outdir = '/Users/olgabot/botryllus/adhoc-analysis/2022-apr--gather-botryllus-in-human-mouse-with-kmers/'
# ! mkdir $outdir

# Iterate over all botryllus proteins to do `gather` on human and mouse

In [4]:
botryllus_dir = '/Users/olgabot/botryllus/data/botryllus-proteins/'

In [5]:
human_gencode_dir = '/Users/olgabot/botryllus/data/gencode/v38/'
mouse_gencode_dir = '/Users/olgabot/botryllus/data/gencode/M28/'

In [6]:
ls $human_gencode_dir

GRCh38.primary_assembly.genome.fa
GRCh38.primary_assembly.genome.fa.fai
gencode.v38.basic.annotation.gff3
gencode.v38.basic.annotation.gtf.gz
gencode.v38.basic.annotation.mhc_region.gff3
gencode.v38.basic.annotation.mhc_region.gtf
gencode.v38.basic.annotation.protein.fa
gencode.v38.basic.annotation.protein.fa.hp.k24.scale5.sig
gencode.v38.basic.annotation.protein.fa.sig
gencode.v38.chr_patch_hapl_scaff.basic.annotation.gff3.gz
gencode.v38.pc_translations.fa.gz
gencode.v38.pc_translations.fa.gz.sig
mhc_genes.txt
mhc_ids.csv


In [7]:
ls $mouse_gencode_dir

GRCm39.primary_assembly.genome.fa
GRCm39.primary_assembly.genome.fa.fai
gencode.vM28.basic.annotation.gff3
gencode.vM28.basic.annotation.protein.fa
gencode.vM28.basic.annotation.protein.fa.hp.k24.scale10.sig
gencode.vM28.basic.annotation.protein.fa.hp.k24.scale5.sig


## set signature files

In [8]:
human_sigfile = os.path.join(
    human_gencode_dir, "gencode.v38.basic.annotation.protein.fa.hp.k24.scale5.sig"
)
mouse_sigfile = os.path.join(
    mouse_gencode_dir, "gencode.vM28.basic.annotation.protein.fa.hp.k24.scale5.sig"
)

In [9]:
botryllus_sigfile = os.path.join(botryllus_dir, 'Bs_proteins.fa.hp.k24.scale5.sig')

## Load human and mouse signatures as indecies/databases

In [10]:
human_db = sourmash.load_file_as_index(human_sigfile)

In [11]:
mouse_db = sourmash.load_file_as_index(mouse_sigfile)

In [12]:
dbs = {'human': human_db, 'mouse': mouse_db}

## Load human, mouse, botryllus fastas to read in sequences to write matching k-mers

In [13]:
def fasta_to_dict(fasta_filename):
    sequences = {}
    with screed.open(fasta_filename) as records:
        for record in records:
            sequences[record['name']] = record['sequence']
            
    return sequences

In [14]:
botryllus_sequences = fasta_to_dict(os.path.join(botryllus_dir, 'Bs_proteins.fa.gz'))
n_botryllus_seqs = len(botryllus_sequences)
n_botryllus_seqs

72617

In [15]:
fastas = {
    "human": os.path.join(human_gencode_dir, "gencode.v38.basic.annotation.protein.fa"),
    "mouse": os.path.join(
        mouse_gencode_dir, "gencode.vM28.basic.annotation.protein.fa"
    ),
}
fastas

{'human': '/Users/olgabot/botryllus/data/gencode/v38/gencode.v38.basic.annotation.protein.fa',
 'mouse': '/Users/olgabot/botryllus/data/gencode/M28/gencode.vM28.basic.annotation.protein.fa'}

In [16]:
%time sequences = {k: fasta_to_dict(fasta) for k, fasta in fastas.items()}
for species, seqs in sequences.items():
    print(f"{species}, {len(seqs)}")

CPU times: user 880 ms, sys: 113 ms, total: 993 ms
Wall time: 1.08 s
human, 61543
mouse, 46083


# Move code into functions for a separate file to run on all botryllus sequences

## Load botryllus protein signatures

In [17]:
botryllus_sigs = sourmash.load_file_as_signatures(botryllus_sigfile)

## Iterate over all botryllus signatures

In [18]:
csvs_created = glob.glob(os.path.join(outdir, '*.csv'))
n_csvs_created = len(csvs_created)
n_csvs_created

31972

In [ ]:
from single_gather_multi_db import single_gather_multi_db

total = n_botryllus_seqs - n_csvs_created

for query_sig in tqdm(botryllus_sigs, total=total):
    query_seq = botryllus_sequences[query_sig.name]

    single_gather_multi_db(
        query_sig, query_seq, dbs, sequences, human_mouse_homologs, outdir, force=False
    )

g3.t1 frame:1 had no matches                                                                                                   | 0/40645 [00:00<?, ?it/s]
g4.t1 frame:1 had no matches                                                                                        | 6/40645 [00:08<15:57:49,  1.41s/it]
g20.t1 frame:3 had no matches                                                                                       | 7/40645 [00:17<31:53:00,  2.82s/it]
g23.t1 frame:1 had no matches                                                                                      | 23/40645 [00:29<13:05:58,  1.16s/it]
g25.t1 frame:1 had no matches                                                                                      | 26/40645 [00:40<17:48:38,  1.58s/it]
g29.t1 frame:1 had no matches                                                                                      | 28/40645 [00:48<22:22:09,  1.98s/it]
g40.t1 frame:1 had no matches                                               

## Try joblib parallel

In [ ]:
# from single_gather_multi_db import single_gather_multi_db


# def get_query_seq_and_do_gather(query_sig, force=True):
#     query_seq = botryllus_sequences[query_sig.name]
#     single_gather_multi_db(
#         query_sig, query_seq, dbs, sequences, human_mouse_homologs, outdir, force=force
#     )


# Parallel(n_jobs=16)(
#     delayed(get_query_seq_and_do_gather)(query_sig)
#     for query_sig in tqdm(botryllus_sigs, total=n_botryllus_seqs)
# )

In [ ]:
query_sig.minhash.hashes

In [38]:
query_sig.minhash.hashes

{26845412769272181: 1, 130721713846454352: 1, 131914162857596854: 1, 264129614277670437: 1, 326882603727633742: 1, 377565399683672287: 1, 649124533855795895: 1, 708849019351108677: 1, 750087187631575148: 1, 946277241446369080: 1, 977543989306394153: 1, 1087005273513554815: 1, 1123700807937130837: 1, 1269212021692616851: 1, 1367492085946631369: 1, 1372922037674299249: 1, 1854082084171101908: 1, 1930249545546361252: 1, 2268059299364738368: 1, 2312820076463029581: 1, 2403321648259680602: 1, 2501993514450748528: 1, 2547265834897203550: 1, 2654618888434498289: 1, 2704609051787278683: 1, 2734346087673125713: 1, 2735381053715299187: 1, 2838016961584743320: 1, 2876835207078369075: 1, 3050583332190630828: 1, 3117881101863706478: 1, 3174372614671418122: 1, 3273385247373028685: 1, 3351888505178207671: 1, 3420126247547612638: 1, 3477421962320830521: 1, 3600067912142119623: 1}